<a href="https://colab.research.google.com/github/luck058/yfinance-prediction/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
!git clone https://github.com/luck058/yfinance-prediction

%cd yfinance-prediction

Cloning into 'yfinance-prediction'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 72 (delta 43), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 60.09 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/yfinance-prediction


In [3]:
%run import_data.ipynb
help()

# Set up data fetcher using:
data_fetcher = YahooFinanceDataFetcher(self, ticker_symbol, period="max", interval="1d")

# Get historical data using:
df_data = data_fetcher.get_data(verbose=False)

# Get financials using:
df_financials = data_fetcher.get_financials(verbose=False)

# Get actions using:
df_actions = data_fetcher.get_actions(verbose=False)



In [4]:
# Defined in "import_data.ipynb"
data_fetcher = YahooFinanceDataFetcher("AAPL", period="max", interval="1d")

X, y = data_fetcher.get_X_y(reload=True)

display(X.head())
print(X.shape)

display(y.head())
print(y.shape)

,High,Low,Close,Volume,Dividends,Stock Splits,Open-0,Open-1,Open-2,Open-4,...,Low max-10,Close max-10,Open-0 min-10,High min-10,Low min-10,Close min-10,Open-0 mean-10,High mean-10,Low mean-10,Close mean-10
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-26 00:00:00-05:00,0.122469,0.122039,0.122039,55574400,0.0,0.0,0.122039,0.111726,0.106140,0.097116,...,0.122039,0.122039,0.087232,0.087232,0.086802,0.086802,0.099952,0.100296,0.099866,0.099866
1980-12-29 00:00:00-05:00,0.124188,0.123758,0.123758,93161600,0.0,0.0,0.123758,0.122039,0.111726,0.101842,...,0.123758,0.123758,0.087232,0.087232,0.086802,0.086802,0.102444,0.102788,0.102358,0.102358
1980-12-30 00:00:00-05:00,0.121180,0.120750,0.120750,68880000,0.0,0.0,0.121180,0.123758,0.122039,0.106140,...,0.123758,0.123758,0.087232,0.087232,0.086802,0.086802,0.105151,0.105495,0.105065,0.105065
1980-12-31 00:00:00-05:00,0.117742,0.117313,0.117313,35750400,0.0,0.0,0.117742,0.121180,0.123758,0.111726,...,0.123758,0.123758,0.088951,0.089381,0.088951,0.088951,0.108202,0.108546,0.108116,0.108116
1981-01-02 00:00:00-05:00,0.119461,0.118602,0.118602,21660800,0.0,0.0,0.118602,0.117742,0.121180,0.122039,...,0.123758,0.123758,0.091530,0.091959,0.091530,0.091530,0.111167,0.111554,0.111082,0.111082


(8893, 59)


,Open
Date,
1980-12-26 00:00:00-05:00,0.123758
1980-12-29 00:00:00-05:00,0.121180
1980-12-30 00:00:00-05:00,0.117742
1980-12-31 00:00:00-05:00,0.118602
1981-01-02 00:00:00-05:00,0.116453


(8893,)


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso



In [6]:
def evaluate_model(model, X_train, y_train, other_scoring=[], return_estimator=False):
    cv_results = cross_validate(model, X_train, y_train, scoring=["r2", "neg_mean_squared_error"]+other_scoring, cv=10, return_estimator=return_estimator)
    print(f"R2: {print_cv_results(cv_results['test_r2'])}")
    print(f"MSE: {print_cv_results(cv_results['test_neg_mean_squared_error'] * (-1))}")
    for scoring in other_scoring:
        print(f"{scoring}: {print_cv_results(cv_results[f'test_{scoring}'])}")
    print()
    return cv_results

In [33]:
def print_cv_results(results, z=1.96):
    """Prints the mean with confidence intervals (z-score can be modified, default 95% confidence)"""
    mean = np.mean(results)
    std_err = np.std(results) / np.sqrt(len(results))
    return f"{round(mean, 4)} " + u"\u00B1" + f"{round(z * std_err, 4)}"

In [34]:
print(f"Model: LinearRegression")
linear_regression = evaluate_model(LinearRegression(), X, y, return_estimator=True)
display(pd.DataFrame(zip(X.columns, linear_regression["estimator"][0].coef_.round(3))).sort_values(by=1))


Model: LinearRegression
R2: 0.9969 ±0.0013
MSE: 0.008 ±0.0112



,0,1
28,Dividends-2,-1.004
57,Low mean-10,-0.372
6,Open-0,-0.339
58,Close mean-10,-0.257
29,Dividends-4,-0.227
20,Close-2,-0.215
43,Open-0 mean-3,-0.163
19,Close-1,-0.162
14,High-8,-0.154
41,Low min-3,-0.133


In [35]:
print(linear_regression)

{'fit_time': array([0.0462184 , 0.06299973, 0.07675052, 0.09183168, 0.13188863,
       0.07304955, 0.10675168, 0.10427189, 0.05959249, 0.08253026]), 'score_time': array([0.00695014, 0.00994921, 0.01493883, 0.0156095 , 0.0071156 ,
       0.00945997, 0.01287222, 0.00691104, 0.00751114, 0.01277828]), 'estimator': [LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression()], 'test_r2': array([0.99684233, 0.99874935, 0.99255849, 0.99688047, 0.99607378,
       0.99448717, 0.99924507, 0.99760285, 0.9992414 , 0.99777203]), 'test_neg_mean_squared_error': array([-4.14590512e-06, -1.00359376e-05, -2.04922683e-05, -2.20393748e-05,
       -1.87928947e-05, -2.87445654e-04, -9.83078178e-05, -3.90982336e-03,
       -1.50986512e-02, -6.03829981e-02])}


In [36]:
class PredictColumn:
    def __init__(self, column):
        self.column = column

    def fit(self, X, y):
        pass

    def predict(self, X):
        return X[self.column]

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    def get_params(self, deep=True):
        return {}

In [37]:
class PredictClose(PredictColumn):
    def __init__(self):
        super().__init__("Close")

In [38]:
class PredictOpen(PredictColumn):
    def __init__(self):
        super().__init__("Open-0")

In [39]:
print(f"Model: PredictOpen")
evaluate_model(PredictOpen(), X, y)

Model: PredictOpen
R2: 0.9914 ±0.0041
MSE: 0.021 ±0.0292



{'fit_time': array([0.00844383, 0.00434613, 0.00424552, 0.00502443, 0.00447297,
        0.00446081, 0.00450969, 0.00476623, 0.00477266, 0.00464439]),
 'score_time': array([0.00227571, 0.00205517, 0.00228906, 0.00212288, 0.00213194,
        0.00208354, 0.00206137, 0.00210547, 0.00213218, 0.00206804]),
 'test_r2': array([0.99145805, 0.99554161, 0.97348055, 0.9893755 , 0.99078042,
        0.99020845, 0.99800148, 0.99317723, 0.99797301, 0.99419108]),
 'test_neg_mean_squared_error': array([-1.12152555e-05, -3.57765761e-05, -7.30286752e-05, -7.50618538e-05,
        -4.41295628e-05, -5.10543100e-04, -2.60251237e-04, -1.11281650e-02,
        -4.03435425e-02, -1.57434747e-01])}

In [40]:
print(f"Model: PredictClose")
evaluate_model(PredictClose(), X, y)

Model: PredictClose
R2: 0.9973 ±0.0012
MSE: 0.0075 ±0.0104



{'fit_time': array([0.01239324, 0.0043416 , 0.00420928, 0.00440407, 0.00406694,
        0.00404072, 0.00399685, 0.0040009 , 0.00395155, 0.0040319 ]),
 'score_time': array([0.00294924, 0.00197268, 0.00194168, 0.00196052, 0.00193882,
        0.00179315, 0.0017705 , 0.00192785, 0.00182247, 0.00196671]),
 'test_r2': array([0.99755743, 0.99943307, 0.99279087, 0.99714232, 0.99657809,
        0.99513726, 0.99928195, 0.99783077, 0.99923329, 0.99792989]),
 'test_neg_mean_squared_error': array([-3.20699575e-06, -4.54937646e-06, -1.98523534e-05, -2.01894516e-05,
        -1.63789824e-05, -2.53549000e-04, -9.35060550e-05, -3.53807850e-03,
        -1.52600833e-02, -5.61045456e-02])}

In [41]:
print(f"Model: Lasso")
alphas = [0.01]
best_error_lasso = -100
best_alpha_lasso = None
best_estimator_lasso = None
for alpha in alphas:
    print(f"alpha: {alpha}")
    lasso_results = evaluate_model(Lasso(alpha=alpha, max_iter=100000), X, y, return_estimator=True)
    if lasso_results["test_neg_mean_squared_error"].mean() > best_error_lasso:
        best_error_lasso = lasso_results["test_neg_mean_squared_error"].mean()
        best_alpha_lasso = alpha
        best_estimator_lasso = lasso_results["estimator"][0]

print("best_alpha:", best_alpha_lasso)
print("Best MSE:", -best_error_lasso.round(2))
display(pd.DataFrame(zip(X.columns, best_estimator_lasso.coef_.round(3))).sort_values(by=1))

Model: Lasso
alpha: 0.01


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.366e+01, tolerance: 3.462e+01
  model = cd_fast.enet_coordinate_descent(


R2: 0.9965 ±0.0017
MSE: 0.0075 ±0.0104

best_alpha: 0.01
Best MSE: 0.01


,0,1
0,High,0.000
30,Dividends-8,0.000
31,Stock Splits-1,-0.000
32,Stock Splits-2,0.000
33,Stock Splits-4,0.000
34,Stock Splits-8,-0.000
35,Open-0 max-3,0.000
37,Low max-3,0.000
38,Close max-3,0.000
39,Open-0 min-3,0.000
